In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tqdm.notebook import tqdm
import time
import urllib.request
import requests
import urllib
import re

In [4]:
# import tkinter
# from datetime import datetime
# from tkinter import ttk
# import tkinter.messagebox
# root = tkinter.Tk()
# root.title("맛집을 찾아줘")
# root.geometry('200x80')

In [5]:
station_list = ['홍대입구역', '강남역', '명동역', '신촌역', '서울대입구역', '사당역', '혜화역', '가산디지털단지역', '서울역', '청담역', '종로3가역', '을지로입구역', '동대문역사문화공원역',   '성신여대입구역', '이태원역', '성수역',  '영등포역', '여의도역', '잠실역', '압구정역', '신사역', '판교역']
def input_keys():
    global stop_check
    key = input("s를 누르면 STOP : ")
    if key == ('s' or 'S'):
        stop_check = 1

# 검색 키워드를 받는 함수
def initkeyword(driver):
    print("더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.")
    driver.get("https://map.naver.com/v5/search/")
    while True:
        key = input("로그인이 완료됐으면 'Y'를 입력해주세요. :")
        if key == 'Y' or key == 'y':
            #print('검색 키워드 입력')
            #encText = urllib.parse.quote(f"{input('검색 키워드 : ')}")
            encText = urllib.parse.quote(station_list[0] + ' 음식점')
            driver.get("https://map.naver.com/v5/search/" + encText)
            break
        else:
            print('잘못된 입력. 다시 실행해주세요.')
            break
    return

# 맨 처음 스크롤을 끝까지 내려서 한 페이지 내 모든 음식점 HTML 태그 불러옴.
def init_scrollOnce(driver):
    driver.switch_to.default_content()
    next_iframe = driver.find_element(By.ID,"searchIframe")
    driver.switch_to.frame(next_iframe)

    for idx in tqdm(range(1,54)):
        try:
            ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
            driver.execute_script("arguments[0].scrollIntoView(true);",ele);
            driver.implicitly_wait(1)
        except:
            continue
    return 


# 가져올 장소
홍대입구역, 강남역, 명동역, 신촌역, 서울대입구역, 사당역, 혜화역, 가산디지털단지역, 서울역, 청담역, 종로3가역, 을지로입구역, 동대문역사문화공원역,   성신여대입구역, 이태원역, 성수역,  영등포역, 여의도역, 잠실역, 압구정역, 신사역
# 가져올 항목
이름, 역, 구, 카테고리, 방문자리뷰갯수, 블로그리뷰갯수, 평점, 주소, 영업시간, 부가서비스, 방문자리뷰글

In [6]:
station_list = ['홍대입구역', '강남역', '명동역', '신촌역', '서울대입구역', '사당역', '혜화역', '가산디지털단지역', '서울역', '청담역', '종로3가역', '을지로입구역', '동대문역사문화공원역',   '성신여대입구역', '이태원역', '성수역',  '영등포역', '여의도역', '잠실역', '압구정역', '신사역', '판교역']
shop_name_list = []
close_station_list = []
gu_list = []
category_list = []
visit_review_list = []
blog_review_list = []
rating_list = []
address_list = []
open_time_list = []
service_list = []
review_contents_list = []

In [9]:
options = webdriver.ChromeOptions()
prefs = {'download.default_directory':'/home/aquila/dev_ws/EDA/data',
         'download.prompt_for_download':False}
options.add_experimental_option('prefs', prefs)
url = "https://map.naver.com/p?c=15.00,0,0,0,dh"
driver = webdriver.Chrome(service=Service("/home/aquila/dev_ws/EDA/driver/chromedriver"),
                          options=options)
driver.get(url)
# 역 별 음식점 검색
input_tab = driver.find_element(By.CLASS_NAME, "input_search")
input_tab.send_keys(station_list[0] + " 음식점")
input_tab.send_keys(Keys.RETURN)

In [10]:
# 새로운 창으로 이동
driver.switch_to.default_content()
next_iframe = driver.find_element(By.ID,"searchIframe")
driver.switch_to.frame(next_iframe)
# 음식점 링크 클릭 후 상세창 열기(li[idx]에서 idx 변경(1~53))
ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/a''');
driver.execute_script("arguments[0].scrollIntoView(true);",ele);
ele.click()

In [11]:
# 리뷰 클릭
review = driver.find_element(By.CLASS_NAME, 'place_fixed_maintab')
i = 1
while True:
    review1 = review.find_element(By.XPATH, f'''//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{i}]''')
    if review1.text == "리뷰":
        break
    else: i+=1
review1.click()
#펼치기
spread = driver.find_element(By.CLASS_NAME, 'Tvx37')
while True:
    try:
        spread.click()
    except:
        break
    
time.sleep(0.3)
review2 = driver.find_element(By.CLASS_NAME, 'uNsI9')
# 자료 전처리
reviews = review2.text
reviews = reviews.split('\n')
to_remove = ["이 키워드를 선택한 인원"]
review_list = list(filter(lambda item: item not in to_remove, reviews))
dic1 = review_list[::2]
dic2 = review_list[1::2]
review_dic = dict(zip(dic1, dic2))
review_dic

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".place_fixed_maintab"}
  (Session info: chrome=116.0.5845.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55c325863e23 <unknown>
#1 0x55c32558c7a7 <unknown>
#2 0x55c3255cb1d3 <unknown>
#3 0x55c3255cb2c1 <unknown>
#4 0x55c325606a04 <unknown>
#5 0x55c3255ec03d <unknown>
#6 0x55c325604369 <unknown>
#7 0x55c3255ebde3 <unknown>
#8 0x55c3255bfa7b <unknown>
#9 0x55c3255c081e <unknown>
#10 0x55c325825638 <unknown>
#11 0x55c325829507 <unknown>
#12 0x55c325833c4c <unknown>
#13 0x55c32582a136 <unknown>
#14 0x55c3257f89cf <unknown>
#15 0x55c32584db98 <unknown>
#16 0x55c32584dd68 <unknown>
#17 0x55c32585ccb3 <unknown>
#18 0x7f3602c94b43 <unknown>


In [17]:
import time  

# 이름, 카테고리
driver.switch_to.default_content()
next_iframe = driver.find_element(By.ID,"entryIframe")
driver.switch_to.frame(next_iframe)

driver.execute_script("window.scrollTo(0, 0)")

shop_box = driver.find_element(By.CLASS_NAME, 'YouOG')
shop_name =  shop_box.find_element(By.CLASS_NAME, "Fc1rA").text
category = shop_box.find_element(By.CLASS_NAME, 'DJJvD').text 
print(shop_name)
print(category)

# 평점, 방문자리뷰갯수, 블로그리뷰갯수
review_box = driver.find_elements(By.CLASS_NAME, 'PXMot')
if len(review_box) > 2:
    rating = review_box[0].find_element(By.TAG_NAME, 'em').text
    visit_review = review_box[1].find_element(By.TAG_NAME, 'em').text
    blog_review = review_box[2].find_element(By.TAG_NAME, 'em').text
else:
    rating = ''
    visit_review = review_box[0].find_element(By.TAG_NAME, 'em').text
    blog_review = review_box[1].find_element(By.TAG_NAME, 'em').text
print(rating)
print(visit_review)
print(blog_review)

# 주소, 구, 역, 영업시간, 부가서비스
address = driver.find_element(By.CLASS_NAME, 'LDgIH').text
print(address)

gu = address.split()[1]
print(gu)

station_box = driver.find_element(By.CLASS_NAME, 'nZapA')
close_station = station_box.find_element(By.TAG_NAME, 'strong').text
print(close_station)

time_btn_box = driver.find_element(By.CLASS_NAME, 'y6tNq')
time_btn = time_btn_box.find_element(By.CLASS_NAME, '_UCia')
time_btn.click()

open_time = []
time_box = driver.find_elements(By.CLASS_NAME, 'A_cdD')
for i in range(1, len(time_box)):
    if '-' in time_box[i].text or ':' in time_box[i].text:
        open_time.append(time_box[i].text)
open_time = str(open_time).replace('\\n', ' ').replace('[', '').replace(']', '').replace("'", '')
print(open_time)

'''
open_time = ''
time_table = driver.find_elements(By.CLASS_NAME, 'w9QyJ')
for day_time in time_table:
    open_time += day_time.text
    open_time = open_time.replace('접기', '')
print(open_time)
'''

services = driver.find_elements(By.CLASS_NAME, 'vV_z_')
service_words = []
for service in services:
    service = service.text 
    if '단체석' in service:
        service_words.append('단체석')
    if '포장' in service:
        service_words.append('포장')
    if '예약' in service:
        service_words.append('예약')
    if '배달' in service:
        service_words.append('포장')
    if '무선 인터넷' in service:
        service_words.append('무선 인터넷')
    if '남/녀 화장실 구분' in service:
        service_words.append('남/녀 화장실 구분')
service_words = str(service_words).replace('[', '').replace(']', '').replace("'", "")
print(service_words)        

# 리뷰 클릭
review = driver.find_element(By.CLASS_NAME, 'place_fixed_maintab')
i = 1
while True:
    review1 = review.find_element(By.XPATH, f'''//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{i}]''')
    if review1.text == "리뷰":
        break
    else: i+=1
review1.click()
#펼치기
time.sleep(1)
spread = driver.find_element(By.CLASS_NAME, 'Tvx37')
while True:
    try:
        spread.click()
    except:
        break
    
time.sleep(0.3)
review2 = driver.find_element(By.CLASS_NAME, 'uNsI9')
# 자료 전처리
reviews = review2.text
reviews = reviews.split('\n')
to_remove = ["이 키워드를 선택한 인원"]
review_list = list(filter(lambda item: item not in to_remove, reviews))
dic1 = review_list[::2]
dic2 = review_list[1::2]
review_dic = dict(zip(dic1, dic2))
review = str(review_dic).replace("{", '').replace('}', '').replace('"', '').replace("'", '')

print(review)

# 음식점 선택 프레임으로 iframe 이동
driver.switch_to.default_content()
next_iframe = driver.find_element(By.ID,"searchIframe")
driver.switch_to.frame(next_iframe)
# 역 별 음식점 검색
driver.switch_to.default_content()
input_tab = driver.find_element(By.CLASS_NAME, "input_search")
btn_clear  = driver.find_element(By.CLASS_NAME, 'btn_clear')
btn_clear.click()
time.sleep(1)
input_tab.send_keys(station_list[1] + " 음식점")
input_tab.send_keys(Keys.RETURN)
time.sleep(1)

#close_time = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > div > div.O8qbU.pSavy > div > a > div:nth-child(2) > div > span._UCia > svg')   
#close_time.click()
        
# 리뷰 클릭
# menu_bar = driver.find_elements(By.CLASS_NAME, 'class="tpj9w _tab-menu')
# if len(menu_bar) == 4:
#     review_tab = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(3)')
#     review_tab.click()
# else:
#      review_tab = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(4)')
#      review_tab.click()
# time.sleep(2) 
# review_btn = driver.find_element(By.CLASS_NAME, 'Tvx37')
# while True:
#     try:
#         review_btn.click()
#     except:
#         break



정화
한식
4.29
448
309
서울 마포구 양화로8길 38 서교동빌딩 1층
마포구
합정역
금 11:30 - 24:00 14:00 - 17:00 브레이크타임, 토 16:00 - 24:00, 일 16:00 - 22:00, 월 11:30 - 23:00 14:00 - 17:00 브레이크타임, 화 11:30 - 23:00 14:00 - 17:00 브레이크타임, 수 11:30 - 23:00 14:00 - 17:00 브레이크타임, 목 11:30 - 23:00 14:00 - 17:00 브레이크타임
단체석, 예약
음식이 맛있어요: 252, 친절해요: 187, 재료가 신선해요: 109, 특별한 메뉴가 있어요: 100, 가성비가 좋아요: 53, 양이 많아요: 50, 단체모임 하기 좋아요: 50, 매장이 청결해요: 48, 특별한 날 가기 좋아요: 26, 매장이 넓어요: 23, 인테리어가 멋져요: 18, 혼밥하기 좋아요: 8, 화장실이 깨끗해요: 8, 직접 잘 구워줘요: 7, 고기 질이 좋아요: 6, 메뉴 구성이 알차요: 6, 뷰가 좋아요: 4, 음식이 빨리 나와요: 3, 주차하기 편해요: 3, 반찬이 잘 나와요: 2, 잡내가 적어요: 1, 기본 안주가 좋아요: 1, 건강한 맛이에요: 1


## 최종 코드

In [30]:
driver.close()

In [31]:
global stop_check
from threading import Thread


shop_name_list = []
close_station_list = []
gu_list = []
category_list = []
visit_review_list = []
blog_review_list = []
rating_list = []
address_list = []
open_time_list = []
service_list = []
review_contents_list = []
detailed_review = []
latitude = []
longitude = []

driver = webdriver.Chrome(service=Service("../../driver/chromedriver"))

def main_action():
    
    
    global stop_check
    stop_check = 0
    
    for i in range(len(station_list)):
        # 다음 역 근처 음식점 검색
        driver.switch_to.default_content()
        input_tab = driver.find_element(By.CLASS_NAME, "input_search")
        input_clear  = driver.find_element(By.CLASS_NAME, 'btn_clear')
        input_clear.click()
        time.sleep(0.5)
        input_tab.send_keys(station_list[i] + " 음식점")
        input_tab.send_keys(Keys.RETURN)
        time.sleep(0.5)
        
        while stop_check != 1:    
            for idx in tqdm(range(1,54)):
                # 음식점 선택 프레임으로 iframe 이동
                driver.switch_to.default_content()
                next_iframe = driver.find_element(By.ID,"searchIframe")
                driver.switch_to.frame(next_iframe)
                
                # 음식점 링크 클릭 후 상세창 열기(li[idx]에서 idx 변경(1~53))
                ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
                driver.execute_script("arguments[0].scrollIntoView(true);",ele);
                #ele.click()
                WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''')))
                ele.click()
                
                # 상세창으로 이동
                time.sleep(1)
                driver.implicitly_wait(3)
                driver.switch_to.default_content()
                next_iframe = driver.find_element(By.ID,"entryIframe")
                driver.switch_to.frame(next_iframe)
                driver.execute_script("window.scrollTo(0, 0)")
                driver.implicitly_wait(3)
                time.sleep(1)
                
                # 이름, 카테고리
                driver.execute_script("window.scrollTo(0, 0)")

                shop_box = driver.find_element(By.CLASS_NAME, 'YouOG')
                shop_name =  shop_box.find_element(By.CLASS_NAME, "Fc1rA").text
                category = shop_box.find_element(By.CLASS_NAME, 'DJJvD').text 
                #print(shop_name)
                #print(category)

                # 평점, 방문자리뷰갯수, 블로그리뷰갯수
                review_box = driver.find_elements(By.CLASS_NAME, 'PXMot')
                if len(review_box) > 2:
                    rating = review_box[0].find_element(By.TAG_NAME, 'em').text
                    visit_review = review_box[1].find_element(By.TAG_NAME, 'em').text
                    blog_review = review_box[2].find_element(By.TAG_NAME, 'em').text
                else:
                    rating = ''
                    visit_review = review_box[0].find_element(By.TAG_NAME, 'em').text
                    blog_review = review_box[1].find_element(By.TAG_NAME, 'em').text
                #print(rating)
                #print(visit_review)
                #print(blog_review)

                # 주소, 구, 역, 영업시간, 부가서비스
                address = driver.find_element(By.CLASS_NAME, 'LDgIH').text
                #print(address)

                gu = address.split()[1]
                #print(gu)

                station_box = driver.find_element(By.CLASS_NAME, 'nZapA')
                close_station = station_box.find_element(By.TAG_NAME, 'strong').text
                #print(close_station)

                time_btn_box = driver.find_element(By.CLASS_NAME, 'y6tNq')
                time_btn = time_btn_box.find_element(By.CLASS_NAME, '_UCia')
                time_btn.click()

                open_time = []
                time_box = driver.find_elements(By.CLASS_NAME, 'A_cdD')
                for i in range(1, len(time_box)):
                    if '-' in time_box[i].text or ':' in time_box[i].text:
                        open_time.append(time_box[i].text)
                open_time = str(open_time).replace('\\n', ' ').replace('[', '').replace(']', '').replace("'", '')
                #print(open_time)

                services = driver.find_elements(By.CLASS_NAME, 'vV_z_')
                service_words = []
                for service in services:
                    service = service.text 
                    if '단체석' in service:
                        service_words.append('단체석')
                    if '포장' in service:
                        service_words.append('포장')
                    if '예약' in service:
                        service_words.append('예약')
                    if '배달' in service:
                        service_words.append('포장')
                    if '무선 인터넷' in service:
                        service_words.append('무선 인터넷')
                    if '남/녀 화장실 구분' in service:
                        service_words.append('남/녀 화장실 구분')
                service_words = str(service_words).replace('[', '').replace(']', '').replace("'", "")
                #print(service_words)     
                
                            # 리뷰 클릭
                review = driver.find_element(By.CLASS_NAME, 'place_fixed_maintab')
                i = 1
                while True:
                    review1 = review.find_element(By.XPATH, f'''//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{i}]''')
                    if review1.text == "리뷰":
                        break
                    else: i+=1
                review1.click()
                #펼치기
                time.sleep(1)
                spread = driver.find_element(By.CLASS_NAME, 'Tvx37')
                while True:
                    try:
                        spread.click()
                    except:
                        break
                    
                time.sleep(0.3)
                review2 = driver.find_element(By.CLASS_NAME, 'uNsI9')
                # 자료 전처리
                reviews = review2.text
                reviews = reviews.split('\n')
                to_remove = ["이 키워드를 선택한 인원"]
                review_list = list(filter(lambda item: item not in to_remove, reviews))
                dic1 = review_list[::2]
                dic2 = review_list[1::2]
                review_dic = dict(zip(dic1, dic2))
                review = str(review_dic).replace("{", '').replace('}', '').replace('"', '').replace("'", '')
                #print(review)
                
                # 개인 리뷰
                detail_re1 = driver.find_element(By.CLASS_NAME, 'eCPGL') # 리뷰 상위 
                detail_re3 = driver.find_element(By.CLASS_NAME, 'fvwqf') # 전체리뷰 더보기
                #전체 리뷰 펼치기
                while True:
                    detail_re2 = detail_re1.find_elements(By.CLASS_NAME, 'YeINN') # 전체 리뷰
                    try:
                        detail_re3.click()
                        if len(detail_re2) >=100:
                            break
                    except:
                        break
                #단일 리뷰 펼치기
                driver.execute_script("window.scrollTo(0, 0);")
                detail_re4 = detail_re1.find_elements(By.CLASS_NAME, 'rvCSr')
                a = 1
                while True:
                    detail_re4[a].click()
                    driver.implicitly_wait(3)
                    a+=1
                    if a>=len(detail_re4):
                        break
                    
                # 리뷰 가져오기(빈 글은 무시하고)
                all_detail = []
                b = 0
                while b < len(detail_re2):
                    try:
                        # Check if b is a valid index
                        if b >= 0 and b < len(detail_re2):
                            detail_re5 = detail_re2[b].find_element(By.CLASS_NAME, 'xHaT3')
                            all_detail.append(detail_re5.text)
                    except NoSuchElementException:
                        # Handle the case when the element is not found (optional)
                        pass
                    
                    b += 1
                
                shop_name_list.append(shop_name)
                close_station_list.append(close_station)
                gu_list.append(gu)
                category_list.append(category)
                visit_review_list.append(visit_review)
                blog_review_list.append(blog_review)
                rating_list.append(rating)
                address_list.append(address)
                open_time_list.append(open_time)
                service_list.append(service_words)
                review_contents_list.append(review)
                detailed_review.append(all_detail)

                print(shop_name, close_station, gu, category, visit_review, blog_review, rating, address, open_time, service_words, review)
    
            try:
                next_page = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div[4]/div[2]/a[7]''')))
                usable = next_page.get_attribute('aria-disabled')                                                                                              
                if usable == 'false':
                    ActionChains(driver).move_to_element(next_page).perform()
                    next_page.send_keys(Keys.ENTER)
                    # next_page.click()
                    driver.implicitly_wait(2)
                else:
                    print('Sucessfully Complete !!')
                    break
            except:
                break
                
            usable = next_page.get_attribute('aria-disabled')                                                                                              
            
            
            # if usable == 'false':
            #     ActionChains(driver).move_to_element(next_page).perform()
            #     next_page.send_keys(Keys.ENTER)
            #     # next_page.click()
            #     driver.implicitly_wait(2)
            # else:
            #     print('Sucessfully Complete !!')
            #     break
    return 

from pynput import keyboard
def on_press(key):
    print('Key %s pressed' % key)

def on_release(key):
    global stop_check
    # print('Key %s released' %key)
    if key == keyboard.Key.esc: #esc 키가 입력되면 종료
        stop_check = 1
        return False

# def stop_func():
#     with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#         listener.join()        
        
import sys
if __name__ == '__main__':
    initkeyword(driver)
    driver.implicitly_wait(2)
    init_scrollOnce(driver)
    print('종료를 원하면 esc를 누르세요.')
    #t2 = Thread(target=main_action())
    main_action()
    # t1 = Thread(target=stop_func())
    # with keyboard.Listener(
    #     on_press=on_press,
    #     on_release=on_release) as listener:
    #     listener.join()    
    # t1.start()
    # t1.join()
    #t2.start()


더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.


  0%|          | 0/53 [00:00<?, ?it/s]

종료를 원하면 esc를 누르세요.


  0%|          | 0/53 [00:00<?, ?it/s]

다옴383 홍대입구역 마포구 양식 910 537 4.8 서울 마포구 동교로38길 27-14 반지층 북쪽 다옴383 금 12:00 - 21:30 15:00 - 16:30 브레이크타임 20:30 라스트오더, 토 12:00 - 21:30 20:30 라스트오더, 일 12:00 - 21:30 20:30 라스트오더, 월 12:00 - 21:30 15:00 - 16:30 브레이크타임 20:30 라스트오더, 화 12:00 - 21:30 15:00 - 16:30 브레이크타임 20:30 라스트오더, 수 12:00 - 21:30 15:00 - 16:30 브레이크타임 20:30 라스트오더, 목 12:00 - 21:30 15:00 - 16:30 브레이크타임 20:30 라스트오더 예약, 무선 인터넷 음식이 맛있어요: 527, 친절해요: 325, 특별한 날 가기 좋아요: 202, 인테리어가 멋져요: 196, 특별한 메뉴가 있어요: 176, 재료가 신선해요: 158, 매장이 청결해요: 126, 가성비가 좋아요: 103, 양이 많아요: 73, 화장실이 깨끗해요: 27, 뷰가 좋아요: 25, 혼밥하기 좋아요: 8, 단체모임 하기 좋아요: 6, 차분한 분위기에요: 4, 음식이 빨리 나와요: 4, 주차하기 편해요: 3, 고기 질이 좋아요: 2, 메뉴 구성이 알차요: 2, 술이 다양해요: 1, 아늑해요: 1, 음료가 맛있어요: 1, 잡내가 적어요: 1, 사진이 잘 나와요: 1, 매장이 넓어요: 1, 직접 잘 구워줘요: 1, 대화하기 좋아요: 1
어사출또 홍대점 홍대입구역 마포구 생선회 180 50  서울 마포구 동교로27길 8 호성빌딩 1층 금 11:30 - 23:30 22:30 라스트오더, 토 11:30 - 23:30 22:30 라스트오더, 월 11:30 - 23:30 22:30 라스트오더, 화 11:30 - 23:30 22:30 라스트오더, 수 11:30 - 23:30 22:30 라스트오더, 목 11:30 - 23:30 22:30 라스트오더 단체석, 포장, 포장, 무선 인터넷, 남/녀 화장실 

  0%|          | 0/53 [00:00<?, ?it/s]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="entryIframe"]"}
  (Session info: chrome=116.0.5845.179); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5601fe4d5e23 <unknown>
#1 0x5601fe1fe7a7 <unknown>
#2 0x5601fe23d1d3 <unknown>
#3 0x5601fe23d2c1 <unknown>
#4 0x5601fe278a04 <unknown>
#5 0x5601fe25e03d <unknown>
#6 0x5601fe276369 <unknown>
#7 0x5601fe25dde3 <unknown>
#8 0x5601fe231a7b <unknown>
#9 0x5601fe23281e <unknown>
#10 0x5601fe497638 <unknown>
#11 0x5601fe49b507 <unknown>
#12 0x5601fe4a5c4c <unknown>
#13 0x5601fe49c136 <unknown>
#14 0x5601fe46a9cf <unknown>
#15 0x5601fe4bfb98 <unknown>
#16 0x5601fe4bfd68 <unknown>
#17 0x5601fe4cecb3 <unknown>
#18 0x7f715df6ab43 <unknown>


In [32]:
print(len(shop_name_list), shop_name_list)
print(len(close_station_list), close_station_list)
print(len(gu_list), gu_list)
print(len(category_list), category_list)
print(len(visit_review_list), visit_review_list)
print(len(blog_review_list), blog_review_list)
print(len(rating_list), rating_list)
print(len(address_list), address_list)
print(len(open_time_list), open_time_list)
print(len(service_list), service_list)
print(len(review_contents_list), review_contents_list)
print(len(detailed_review), detailed_review)

53 ['다옴383', '어사출또 홍대점', '토끼정 홍대', '합플레이스 홍대점', '현우동 홍대', '마크스로프트', '빌라 더 다이닝', '만동제과', '훌리건 커피', '미니히노코', '우와', '감성타코 합정점', '연하동', '리차드하우스 연남', '돈주는남자 본점', '팻투데이', '하우에버 연남', '나비카페', '연어롭다', '작당모의', '우동 카덴', '하이디라오 홍대점', '뚜이연남', '남도뽀글이 홍대점', '오마카세 오사이초밥 홍대점', '스케줄합정', '핵밥 홍대점', '온실', '마포곱창타운 연남점', '카멜로연남', '연교', '육몽 홍대본점', '아오이토리', '신미경홍대닭갈비', '제스티살룬 연남', '연남취향', '뉴욕아파트먼트', '코코발리비비큐', '옥소반 합정점', '로쏘 1924 나폴리 화덕피자 홍대점', '멜팅오브', '또보겠지떡볶이집 붕붕허니비점', '고요베이크샵', '육지', '멘지 홍대점', '이미커피', '피자네버슬립스 합정상수점', '오퍼 카페', '뉴오더클럽 연남', '별자리마라탕&훠궈 무한리필 홍대점', '스파카 나폴리', '진진', '헤이죠지']
53 ['홍대입구역', '홍대입구역', '합정역', '상수역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '합정역', '홍대입구역', '홍대입구역', '홍대입구역', '상수역', '홍대입구역', '신촌역', '홍대입구역', '홍대입구역', '합정역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '합정역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '합정역', '홍대입구역', '합정역', '홍대입구역', '홍대입구역', '홍대입구역', '홍대입구역', '합정역', '홍대입구역', '홍대입구역', '합정역', '홍대입구역', '홍대입구역', '홍대입구역'

In [44]:
import pandas as pd
df = pd.DataFrame({'이름': shop_name_list,
                   '역': close_station_list,
                   '구': gu_list,
                   '카테고리': category_list,
                   '방문자_리뷰_개수': visit_review_list,
                   '블로그_리뷰_개수': blog_review_list,
                   '평점': rating_list,
                   '주소': address_list,
                   '영업시간': open_time_list, 
                   '부가서비스': service_list,
                   '방문자_리뷰_글': review_contents_list,
                   '개인리뷰' : detailed_review
                   })
df

,이름,역,구,카테고리,방문자_리뷰_개수,블로그_리뷰_개수,평점,주소,영업시간,부가서비스,방문자_리뷰_글,개인리뷰
0,다옴383,홍대입구역,마포구,양식,910,537,4.8,서울 마포구 동교로38길 27-14 반지층 북쪽 다옴383,금 12:00 - 21:30 15:00 - 16:30 브레이크타임 20:30 라스트...,"예약, 무선 인터넷","음식이 맛있어요: 527, 친절해요: 325, 특별한 날 가기 좋아요: 202, 인...",[신랑이 식사하고 나오자마자 여태 먹어본 빠네파스타 중 제일 맛있었다고 하네요!! ...
1,어사출또 홍대점,홍대입구역,마포구,생선회,180,50,,서울 마포구 동교로27길 8 호성빌딩 1층,"금 11:30 - 23:30 22:30 라스트오더, 토 11:30 - 23:30 2...","단체석, 포장, 포장, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 112, 재료가 신선해요: 95, 친절해요: 83, 가성비가 좋아...",[오랜만에 어사출또 왔는데 회도 신선하고 맛있어요 ㅎㅎ 역시나 어사출또하면 가성비인...
2,토끼정 홍대,합정역,마포구,일식당,"1,227","1,790",4.53,서울 마포구 잔다리로2길 13 1층~3층,"금 11:30 - 22:00 21:00 라스트오더, 토 11:30 - 22:00 2...",예약,"음식이 맛있어요: 469, 친절해요: 261, 양이 많아요: 160, 특별한 메뉴가...","[맛도 좋고 직원분들도 친절하시고\n사진도 잘나와서 인스타 올리기 좋아요 😋, 토끼..."
3,합플레이스 홍대점,상수역,마포구,한식,41,461,,서울 마포구 잔다리로 6 2층,"금 17:00 - 03:00, 토 17:00 - 05:00, 일 17:00 - 05...","단체석, 포장, 예약, 포장, 무선 인터넷, 남/녀 화장실 구분","음식이 맛있어요: 32, 매장이 넓어요: 26, 특별한 메뉴가 있어요: 23, 술이...","[수육튀김, 마늘폭탄꼬들목, 하이볼 주문했어요❣️ 홍대역 술집 그 중에도 감성 헌팅..."
4,현우동 홍대,홍대입구역,마포구,"우동,소바",55,308,,서울 마포구 와우산로21길 28-8 1층,금 11:30 - 21:30 15:00 - 17:30 브레이크타임 20:30 라스트...,"단체석, 남/녀 화장실 구분","음식이 맛있어요: 50, 친절해요: 30, 매장이 청결해요: 22, 재료가 신선해요...",[홍대 혼밥맛집을 찾다가 미슐랭에 선정된 현우동을 찾았 어요! 덴뿌라붓카케우동이라는...
5,마크스로프트,홍대입구역,마포구,"카페,디저트",102,559,,서울 마포구 와우산로27길 20 1층,"금 09:00 - 21:00, 토 09:00 - 21:00, 월 09:00 - 21...","단체석, 포장, 무선 인터넷","디저트가 맛있어요: 46, 커피가 맛있어요: 43, 인테리어가 멋져요: 36, 음료...",[아아 아몬드크로와상 멜론케이크 먹었어요 카페 공간이 너무 좋아요 음악도 좋고 넓고...
6,빌라 더 다이닝,홍대입구역,마포구,양식,"5,191","1,487",4.73,"서울 마포구 동교로30길 16 JnS.Bldg B1, 1F, 2F, 3F","금 12:00 - 23:00 22:00 라스트오더, 토 12:00 - 23:00 2...","단체석, 포장, 예약, 포장, 무선 인터넷","음식이 맛있어요: 2716, 인테리어가 멋져요: 2099, 친절해요: 1759, 특...","[빌라 더 다이닝 캐주얼 세트 메뉴를 주문했어요! 새우 로 제 파스타, 땡초 까르보..."
7,만동제과,홍대입구역,서대문구,베이커리,"4,621","3,046",4.51,서울 서대문구 연희로 32,"매일 10:00 - 20:30, - 예약 픽업시간 월~금 11:00~17:00","예약, 포장, 무선 인터넷","빵이 맛있어요: 2328, 특별한 메뉴가 있어요: 1055, 친절해요: 1033, ...",[만동제과~넘 맛있는 빵들이 가득 가득하네요. 하나씩 다 담아 오구 싶은 충동이 가...
8,훌리건 커피,홍대입구역,마포구,"카페,디저트",435,"1,275",,서울 마포구 와우산로29바길 11-5 1층,매일 11:00 - 21:30,"단체석, 무선 인터넷, 남/녀 화장실 구분","디저트가 맛있어요: 229, 인테리어가 멋져요: 223, 친절해요: 219, 커피가...",[힙한 홍대 카페입니다👍🏻 사진 찍고 싶은 핫스팟이 한두 군데가 아니에요ㅎㅎ 브런치...
9,미니히노코,홍대입구역,마포구,햄버거,12,68,,서울 마포구 동교로38길 33-16 2층 미니히노코,"금 11:30 - 20:00, 토 11:30 - 20:00, 일 11:30 - 20...","단체석, 포장","인테리어가 멋져요: 6, 친절해요: 6, 매장이 청결해요: 6, 음식이 맛있어요: ...",[연남동에 새로 생긴 후쿠오카 숯불버거 전문점 오픈한 지 얼마 안 되어서 아직 알려...


In [46]:
df.to_csv('hongdea.csv',encoding='utf-8')

In [12]:
global stop_check
from threading import Thread


driver = webdriver.Chrome(service=Service("../../EDA/driver/chromedriver"))
#encText = urllib.parse.quote("마포구 음식점")
# transfer_session()
#driver.get("https://map.naver.com/v5/search/"+encText)
#actions = ActionChains(driver)

shop_name_list = []
close_station_list = []
gu_list = []
category_list = []
visit_review_list = []
blog_review_list = []
rating_list = []
address_list = []
open_time_list = []
service_list = []
review_contents_list = []

def main_action():
    global stop_check
    stop_check = 0
    while stop_check != 1:    
        for idx in tqdm(range(1,54)):
            # 음식점 선택 프레임으로 iframe 이동
            driver.switch_to.default_content()
            next_iframe = driver.find_element(By.ID,"searchIframe")
            driver.switch_to.frame(next_iframe)
            
            # # 별점 4.5, 리뷰 갯수 600개 넘는 것만 맛집으로 판별, 상세창으로 이동
            # try: 
            #     rating_box = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/div/div/span[2]''')
            #     rating = float(rating_box.text.split('\n')[1])

            #     review_box = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/div/div/span[3]''')
            #     # review = int(review_box.text.split('\n')[1])
            #     review = int([review_box.text.split(' ')[1] if '+' not in review_box.text else review_box.text.replace('+','').split(' ')[1]][0])
            # except:
            #     continue
            # if rating >= 4.8 and review >= 900:
            #     prize = 1
            # elif rating >= 4.65 and review >= 700:
            #     prize = 2
            # elif rating >= 4.5 and review >= 500:
            #     prize = 3 
            # if rating < 4.5 or review < 600:
            #     continue
            ele = driver.find_element(By.XPATH,f'''//*[@id="_pcmap_list_scroll_container"]/ul/li[{idx}]/div[1]/a[1]/div/div/span[1]''');
            driver.execute_script("arguments[0].scrollIntoView(true);",ele);
            ele.click()
            # store_len = driver.find_elements(By.CSS_SELECTOR,'.XUrfU div ul li .tzwk0 .bSoi3 .N_KDL')
            # len(store_len)
            # store_len[idx-1].text.split('\n')
            
            time.sleep(1)
            driver.implicitly_wait(3)
            driver.switch_to.default_content()
            next_iframe = driver.find_element(By.ID,"entryIframe")
            driver.switch_to.frame(next_iframe)
            driver.execute_script("window.scrollTo(0, 0)")
            driver.implicitly_wait(3)
            time.sleep(1)
            # star = driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[1]/a''')
            star = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[1]/a''')))
            # By.LINK_TEXT,store_len[0].text.split('\n')[0]                                          

            if star.text == '문의':
                # star = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[2]/a'''))) #driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[2]/a''')
                star = driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[2]/a''')
            if star.text == '거리뷰':
                # star = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[1]/a'''))) #driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[1]/a''')
                star  = driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div/div/div[2]/div[3]/div/span[1]/a''')
            if star.get_attribute('aria-pressed') == 'true':
                continue
            
            
            ActionChains(driver).move_to_element(star).perform()
            star.send_keys(Keys.ENTER)
            # star.send_keys(Keys.ENTER)
            time.sleep(1)
            store_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            store_kind = driver.find_element(By.CLASS_NAME,'DJJvD').text
            # star_kind_web = driver.find_elements(By.CSS_SELECTOR,'.swt-save-group-list .swt-save-group-item') #swt-save-group-name
            # star_kind_list =  [each.text.split('\n')[0] for each in star_kind_web]
            page = driver.page_source
            soup = BeautifulSoup(page,'html.parser')
            # star_kind_web = driver.find_elements(By.CSS_SELECTOR,'.swt-save-group-list .swt-save-group-item') #swt-save-group-name
            star_kind_web = soup.find_all(class_='swt-save-group-item')
            star_kind_list = [re.sub(r"[0-9]", "", each.text.replace('\n','').replace('NEW',''))  for each in star_kind_web]

            driver.implicitly_wait(2)
            time.sleep(1)
            if store_kind not in star_kind_list:
                # new_kind_box = driver.find_element(By.XPATH,'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/button''')
                # new_kind_box.send_keys(Keys.ENTER)
                new_kind_box = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/button'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                ActionChains(driver).move_to_element(new_kind_box).perform()                                         
                new_kind_box.send_keys(Keys.ENTER)  
                driver.implicitly_wait(2)    

                Yellow = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[4]/div/div[3]/div[2]/div[2]/div/button[3]'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                ActionChains(driver).move_to_element(Yellow).perform()                                    
                Yellow.send_keys(Keys.ENTER)
                # if prize == 1: 
                #     Yellow = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[4]/div/div[3]/div[2]/div[2]/div/button[3]'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                #     ActionChains(driver).move_to_element(Yellow).perform()                                    
                #     Yellow.send_keys(Keys.ENTER)
                # elif prize == 2:
                #     Orange = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[4]/div/div[3]/div[2]/div[2]/div/button[2]'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                #     ActionChains(driver).move_to_element(Orange).perform()
                #     Orange.send_keys(Keys.ENTER)
                # elif prize == 3:
                #     Red = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[4]/div/div[3]/div[2]/div[2]/div/button[1]'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                #     ActionChains(driver).move_to_element(Red).perform()
                #     Red.send_keys(Keys.ENTER)          
                # Yellow = driver.find_element(By.XPATH,'''/html/body/div[4]/div/div[3]/div[2]/div[2]/div/button[3]''')
                # Yellow.click()                                           
                name_box = driver.find_element(By.ID,'swt-save-input-folderview-list')
                name_box.send_keys(f'{store_kind}')
                # ok_box = driver.find_element(By.XPATH,'''/html/body/div[4]/div/div[3]/div[3]/button''')
                # ok_box.click()
                ok_box = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[4]/div/div[3]/div[3]/button'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                ActionChains(driver).move_to_element(ok_box).perform()
                ok_box.send_keys(Keys.ENTER)  
            else:
                # Yellow = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,'''/html/body/div[4]/div/div[3]/div[2]/div[2]/div/button[3]'''))) #By.LINK_TEXT,store_len[0].text.split('\n')[0]
                # star_kind_select = driver.find_elements(By.CLASS_NAME,'swt-save-group-item')
                # star_kind_select[star_kind_list.index(store_kind)].click()
                # check_box = star_kind_select[star_kind_list.index(store_kind)]
                check_num = star_kind_list.index(store_kind)
                # star_kind_select = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,f'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/ul/li[{check_num+1}]''')))
                # driver.execute_script("arguments[0].scrollIntoView(true);",star_kind_select); 
                # ActionChains(driver).move_to_element(star_kind_select).perform()
                # star_kind_select.click()
                driver.implicitly_wait(2)
                star_kind_select = driver.find_element(By.XPATH,f'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/ul/li[{check_num+1}]''');
                # star_kind_select = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,f'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/ul/li[{check_num+1}]''')))
                driver.execute_script("arguments[0].scrollIntoView(true);",star_kind_select); 
                ActionChains(driver).move_to_element(star_kind_select).perform()
                star_kind_select.click()
                # star_kind_select = driver.find_element(By.XPATH,f'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/ul/li[{check_num+1}]''');
                # star_kind_select = WebDriverWait(driver, timeout=10).until(EC.element_to_be_clickable((By.XPATH,f'''//*[@id="swt-save-widget-wrap"]/div[2]/div[2]/ul/li[{check_num+1}]''')))
                # button_check = star_kind_select.find_element(By.CSS_SELECTOR,'svg')
                # if button_check.get_attribute('fill') == 'none': 
                #     driver.execute_script("arguments[0].scrollIntoView(true);",star_kind_select); 
                #     ActionChains(driver).move_to_element(star_kind_select).perform()
                #     star_kind_select.click()
                # else:
                #     pass
            driver.implicitly_wait(2)    
            time.sleep(1)
            save_box = driver.find_element(By.CLASS_NAME,'swt-save-btn')
            save_box.send_keys(Keys.ENTER)
            driver.implicitly_wait(2)
        
        # next_page = driver.find_element(By.XPATH,'''//*[@id="app-root"]/div/div[3]/div[2]/a[7]''')
        next_page = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,'''//*[@id="app-root"]/div/div[4]/div[2]/a[7]''')))
        usable = next_page.get_attribute('aria-disabled')                                                                                              
        if usable == 'false':
            ActionChains(driver).move_to_element(next_page).perform()
            next_page.send_keys(Keys.ENTER)
            # next_page.click()
            driver.implicitly_wait(2)
        else:
            print('Sucessfully Complete !!')
            break
    return 

from pynput import keyboard
def on_press(key):
    print('Key %s pressed' % key)

def on_release(key):
    global stop_check
    # print('Key %s released' %key)
    if key == keyboard.Key.esc: #esc 키가 입력되면 종료
        stop_check = 1
        return False

# def stop_func():
#     with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#         listener.join()        
        
import sys
if __name__ == '__main__':
    initkeyword(driver)
    driver.implicitly_wait(2)
    init_scrollOnce(driver)
    print('종료를 원하면 esc를 누르세요.')
    t2 = Thread(target=main_action())
    # t1 = Thread(target=stop_func())
    with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
        listener.join()    
    # t1.start()
    # t1.join()
    t2.start()


더 보기를 누르고 '로그인이 필요합니다' 영역을 눌러 로그인 진행해주세요.
검색 키워드 입력


  0%|          | 0/53 [00:00<?, ?it/s]

종료를 원하면 esc를 누르세요.


  0%|          | 0/53 [00:00<?, ?it/s]

KeyboardInterrupt: 